Query:
```SQL
DELETE FROM DESASTRES_BDE.dbo.DESASTRES_FINAL;
INSERT INTO DESASTRES_BDE.dbo.DESASTRES_FINAL
SELECT x.Cuatrenio, AVG(x.Temperatura) AS Temp_AVG, AVG(x.NivelOxigeno) AS Oxi_AVG,
SUM(x.Tsunamis) AS T_Tsunamis,SUM(x.OlasCalor) AS T_OlasCalor,SUM(x.Terremotos) AS T_Terremotos,
SUM(x.Erupciones) AS T_Erupciones,SUM(x.Incendios) AS T_Incendios,
AVG(x.Muertes_Jovenes) as M_Jovenes_AVG,AVG(x.Muertes_Adultos) as M_Adultos_AVG, AVG(x.Muertes_Ancianos) as M_Ancianos_AVG
FROM
(SELECT CASE WHEN c.anio < 2026 THEN '2023-2026' ELSE '2027-2030' END AS Cuatrenio,
Temperatura =c.temperatura,
NivelOxigeno =c.oxigeno,
Tsunamis= d.Tsunamis,
OlasCalor= d.Olas_calor,
Terremotos= d.Terremotos,
Erupciones= d.Erupciones,
Incendios=d.Incendios, 
Muertes_Jovenes= m.R_Menor15 + m.R_15_a_30,
Muertes_Adultos= m.R_30_a_45 +m.R_45_a_60,
Muertes_Ancianos= m.R_M_a_60
FROM DESASTRES.dbo.clima as c
JOIN DESASTRES.dbo.desastres as d ON c.anio =d.anio
JOIN DESASTRES.dbo.muertes as m ON c.anio = m.anio) x
GROUP BY x.Cuatrenio;
```

In [1]:
import os

from pyspark.sql import SparkSession

driver_path = "/home/coder/working_dir/driver_jdbc/postgresql-42.2.27.jre7.jar"

os.environ['PYSPARK_SUBMIT_ARGS'] = f'--driver-class-path {driver_path} --jars {driver_path} pyspark-shell'
os.environ['SPARK_CLASSPATH'] = driver_path

# Create SparkSession 
spark = SparkSession.builder \
        .master("local") \
        .appName("Conexion entre Pyspark y Postgres") \
        .config("spark.jars", driver_path) \
        .config("spark.executor.extraClassPath", driver_path) \
        .getOrCreate()

In [4]:
# Postgres connection settings
pg_url = "jdbc:postgresql://171.1.1.2:5435/"
pg_user = "postgres" # not recommended to have this value in the code
pg_password = "postgres" # not recommended to have this value in the code
pg_driver = "org.postgresql.Driver"

In [5]:
# Create the connection and read the table
df_clima = spark.read \
    .format("jdbc") \
    .option("url", pg_url+"desastres") \
    .option("dbtable", "clima") \
    .option("user", pg_user) \
    .option("password", pg_password) \
    .option("driver", pg_driver) \
    .load()

df_clima.printSchema()
df_clima.show()

root
 |-- anio: integer (nullable = true)
 |-- temperatura: double (nullable = true)
 |-- oxigeno: double (nullable = true)

+----+-----------+-------+
|anio|temperatura|oxigeno|
+----+-----------+-------+
|2023|       22.5|  230.0|
|2024|       22.7|  228.6|
|2025|       22.9|  227.5|
|2026|       23.1|  226.7|
|2027|       23.2|  226.4|
|2028|       23.4|  226.2|
|2029|       23.6|  226.1|
|2030|       23.8|  225.1|
+----+-----------+-------+



In [6]:
# Create the connection and read the table
df_desastres = spark.read \
    .format("jdbc") \
    .option("url", pg_url+"desastres") \
    .option("dbtable", "desastres") \
    .option("user", pg_user) \
    .option("password", pg_password) \
    .option("driver", pg_driver) \
    .load()

df_desastres.printSchema()
df_desastres.show()

root
 |-- anio: integer (nullable = true)
 |-- tsunamis: integer (nullable = true)
 |-- olas_calor: integer (nullable = true)
 |-- terremotos: integer (nullable = true)
 |-- erupciones: integer (nullable = true)
 |-- incendios: integer (nullable = true)

+----+--------+----------+----------+----------+---------+
|anio|tsunamis|olas_calor|terremotos|erupciones|incendios|
+----+--------+----------+----------+----------+---------+
|2023|       2|        15|         6|         7|       50|
|2024|       1|        12|         8|         9|       46|
|2025|       3|        16|         5|         6|       47|
|2026|       4|        12|        10|        13|       52|
|2027|       5|        12|         6|         5|       41|
|2028|       4|        18|         3|         2|       39|
|2029|       2|        19|         5|         6|       49|
|2030|       4|        20|         6|         7|       50|
+----+--------+----------+----------+----------+---------+



In [7]:
# Create the connection and read the table
df_muertes = spark.read \
    .format("jdbc") \
    .option("url", pg_url+"desastres") \
    .option("dbtable", "muertes") \
    .option("user", pg_user) \
    .option("password", pg_password) \
    .option("driver", pg_driver) \
    .load()

df_muertes.printSchema()
df_muertes.show()

root
 |-- anio: integer (nullable = true)
 |-- r_menor15: integer (nullable = true)
 |-- r_15_a_30: integer (nullable = true)
 |-- r_30_a_45: integer (nullable = true)
 |-- r_45_a_60: integer (nullable = true)
 |-- r_m_a_60: integer (nullable = true)

+----+---------+---------+---------+---------+--------+
|anio|r_menor15|r_15_a_30|r_30_a_45|r_45_a_60|r_m_a_60|
+----+---------+---------+---------+---------+--------+
|2023|     1000|     1300|     1200|     1150|    1500|
|2024|     1200|     1250|     1260|     1678|    1940|
|2025|      987|     1130|     1160|     1245|    1200|
|2026|     1560|     1578|     1856|     1988|    1245|
|2027|     1002|      943|     1345|     1232|     986|
|2028|      957|      987|     1856|     1567|    1756|
|2029|     1285|     1376|     1465|     1432|    1236|
|2030|     1145|     1456|     1345|     1654|    1877|
+----+---------+---------+---------+---------+--------+



In [8]:
# Create temporary tables
df_clima.createOrReplaceTempView("clima")          # clima
df_desastres.createOrReplaceTempView("desastres")  # desastres
df_muertes.createOrReplaceTempView("muertes")      # muertes

In [10]:
df_desastres_final = spark.sql("""
SELECT 
  x.Cuatrenio, 
  AVG(x.Temperatura) AS Temp_AVG, 
  AVG(x.NivelOxigeno) AS Oxi_AVG, 
  SUM(x.Tsunamis) AS T_Tsunamis, 
  SUM(x.OlasCalor) AS T_OlasCalor, 
  SUM(x.Terremotos) AS T_Terremotos, 
  SUM(x.Erupciones) AS T_Erupciones, 
  SUM(x.Incendios) AS T_Incendios, 
  AVG(x.Muertes_Jovenes) as M_Jovenes_AVG, 
  AVG(x.Muertes_Adultos) as M_Adultos_AVG, 
  AVG(x.Muertes_Ancianos) as M_Ancianos_AVG 
FROM 
  (
    SELECT 
      CASE WHEN c.anio < 2026 THEN '2023-2026' ELSE '2027-2030' END AS Cuatrenio, 
      c.temperatura as Temperatura, 
      c.oxigeno as NivelOxigeno, 
      d.Tsunamis as Tsunamis, 
      d.Olas_calor as OlasCalor, 
      d.Terremotos as Terremotos, 
      d.Erupciones as Erupciones, 
      d.Incendios as Incendios, 
      (m.R_Menor15 + m.R_15_a_30) as Muertes_Jovenes, 
      (m.R_30_a_45 + m.R_45_a_60) as Muertes_Adultos, 
      m.R_M_a_60 as Muertes_Ancianos
    FROM 
      clima as c 
      JOIN desastres as d ON c.anio = d.anio 
      JOIN muertes as m ON c.anio = m.anio
  ) x 
GROUP BY 
  x.Cuatrenio;
""")

df_desastres_final.printSchema()
df_desastres_final.show()

root
 |-- Cuatrenio: string (nullable = false)
 |-- Temp_AVG: double (nullable = true)
 |-- Oxi_AVG: double (nullable = true)
 |-- T_Tsunamis: long (nullable = true)
 |-- T_OlasCalor: long (nullable = true)
 |-- T_Terremotos: long (nullable = true)
 |-- T_Erupciones: long (nullable = true)
 |-- T_Incendios: long (nullable = true)
 |-- M_Jovenes_AVG: double (nullable = true)
 |-- M_Adultos_AVG: double (nullable = true)
 |-- M_Ancianos_AVG: double (nullable = true)

+---------+------------------+------------------+----------+-----------+------------+------------+-----------+-------------+------------------+------------------+
|Cuatrenio|          Temp_AVG|           Oxi_AVG|T_Tsunamis|T_OlasCalor|T_Terremotos|T_Erupciones|T_Incendios|M_Jovenes_AVG|     M_Adultos_AVG|    M_Ancianos_AVG|
+---------+------------------+------------------+----------+-----------+------------+------------+-----------+-------------+------------------+------------------+
|2027-2030|23.419999999999998|            

In [16]:
df_desastres_final.write \
    .mode('overwrite') \
    .format("jdbc") \
    .option("url", pg_url+"desastres_bde") \
    .option("dbtable", "desastres_final") \
    .option("user", pg_user) \
    .option("password", pg_password) \
    .option("driver", pg_driver) \
    .save()